In [1]:
# Import Libraries
import sqlite3
import pandas as pd 


In [9]:
# Create a connection with databse
conn = sqlite3.connect('user_hits.db')


In [11]:
# List all tables
pd.read_sql_query("SELECT * FROM sqlite_master;", conn)

,type,name,tbl_name,rootpage,sql
0,table,tbl_user_hits,tbl_user_hits,2,CREATE TABLE [tbl_user_hits]\n (\n...
1,table,tbl_user_geo,tbl_user_geo,4,CREATE TABLE [tbl_user_geo]\n (\n ...
2,table,tbl_user_geo_hits,tbl_user_geo_hits,3,CREATE TABLE [tbl_user_geo_hits]\n ...


In [12]:
# Load tables into dataframe using left join between two tables
df_left_join = pd.read_sql_query(
    "SELECT u.userid, u.date, g.city, g.state \
        FROM tbl_user_hits u lEFT JOIN tbl_user_geo g on u.userid = g.userid;",
      conn
    )
df_left_join.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,2,1/2/2019,None,None
2,3,1/3/2019,El Paso,TX


In [15]:
# using inner join 
df_inner_join = pd.read_sql_query(
    "SELECT u.userid, u.date, g.city, g.state \
        FROM tbl_user_hits u INNER JOIN tbl_user_geo g WHERE u.userid = g.userid;", conn
)
df_inner_join.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,3,1/3/2019,El Paso,TX


In [16]:
# A duplicate of the tbl_user_hits table as datframe
df_user_hits = pd.read_sql_query("SELECT * FROM tbl_user_hits;", conn)
df_user_hits.head()

,userid,date
0,1,1/1/2019
1,2,1/2/2019
2,3,1/3/2019


In [17]:
df_user_geo = pd.read_sql_query("SELECT * FROM tbl_user_geo;", conn)
df_user_geo

,userid,city,state
0,1,Dover,DE
1,3,El Paso,TX
2,5,Ewing,NJ


In [18]:
# close db connection
conn.close()


In [19]:
df_left_join = pd.merge(left=df_user_hits, right=df_user_geo, how='left',
                        left_on='userid', right_on='userid')
df_left_join

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,2,1/2/2019,NaN,NaN
2,3,1/3/2019,El Paso,TX


In [20]:
df_right_join = pd.merge(left=df_user_hits, right=df_user_geo, how='right',
                         left_on='userid', right_on='userid')
df_right_join

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,3,1/3/2019,El Paso,TX
2,5,NaN,Ewing,NJ


In [21]:
# Inner Join using Merge Function 
df_inner_join_mg = pd.merge(left=df_user_hits, right=df_user_geo,
                            left_on='userid', right_on='userid')
df_inner_join_mg

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,3,1/3/2019,El Paso,TX


In [22]:
# Outer Join using merge()
df_outer_join = pd.merge(left=df_user_hits, right=df_user_geo, how='outer',
                         left_on='userid', right_on='userid')
df_outer_join.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,2,1/2/2019,NaN,NaN
2,3,1/3/2019,El Paso,TX
3,5,NaN,Ewing,NJ


## Data Aggregation

In [25]:
# Reopen database connection
conn = sqlite3.connect('user_hits.db')

In [26]:
df_user_geo_hits = pd.read_sql_query("SELECT * FROM tbl_user_geo_hits;", conn)
df_user_geo_hits

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,3,1/1/2019,El Paso,TX
2,1,1/2/2019,Dover,DE
3,2,1/2/2019,Philadelphia,PA
4,3,1/2/2019,El Paso,TX
5,1,1/3/2019,Dover,DE
6,2,1/3/2019,Philadelphia,PA


In [27]:
df_groupby_SQL = pd.read_sql_query("SELECT city, state, COUNT(*) as hits \
                                   FROM tbl_user_geo_hits GROUP BY 1, 2;", conn)
df_groupby_SQL.head()

,city,state,hits
0,Dover,DE,3
1,El Paso,TX,2
2,Philadelphia,PA,2
